In [123]:
import copy
import glob
import numpy as np
import os
import scipy.special as special

from itertools import groupby

ans_path = './ner-mturk/answers*'
ans_file_list = sorted(glob.glob(ans_path))
single_ans_list = sorted(glob.glob('ner-mturk/single_ans*'))
truth_file = './ner-mturk/ground_truth.txt'
path_w = './MBEM_outputs'
alltask_num = 5985
train_num = 5385
train_num5 = 1077
test_num = 600
redundancy = 5
iteration_times = 5
ans_file = single_ans_list[1]

In [4]:
def read_output(path, class_dic_r, to_label=False):
    f = open(path)
    make_xypair = lambda v: list(zip(*[l.strip().split() for l in v]))
    is_emptyline = lambda x: x.strip() != '' and not x.startswith('-DOCSTART-')
    list_int = lambda t: list([class_dic_r[int(v)] for v in t])
    xypairs = [make_xypair(v) for k, v in groupby(f, is_emptyline) if k]
    if to_label == True:
        return ([p[0] for p in xypairs], [list(list_int(p[1])) for p in xypairs])
    if len(xypairs[0]) == 2:
        return ([p[0] for p in xypairs], [list(p[1]) for p in xypairs])
    else:
        return ([p[0] for p in xypairs], [list(p[1:]) for p in xypairs])

In [6]:
def load_crowd_data(ans_file, truth_file, train_num):
    #answers_list[task_num] = [[words, ...], [worker], [[answers]...]]
    answers_list = []
    sentence = []
    answers = []
    worker = []
    words_dic = {}
    get_worker = lambda x: tuple([i for i, l in enumerate(x) if l !='?'])
    for l in open(ans_file):
        ans = l.split()
        if len(l) < 2:
            answers_list.append([sentence, worker, answers])
            sentence = []
            worker = []
            answers = []
        else:
            if len(worker) < 1:
                worker = get_worker(ans[1:])
            if not ans[0] in words_dic:
                words_dic[ans[0]] = len(words_dic)
            sentence.append(ans[0])
            answers.append(get_ans(worker, ans[1:]))            
    #truth_list: [(task_word, answer), ....]
    truth_list = [tuple(l.split()) for l in open(truth_file) if l != "\n"]
    ans_train = answers_list[0:train_num]
#    ans_test = answers_list[train_num:]
    return ans_train, words_dic

In [29]:
def get_ans(workers, answers):
    ans_label = []
    for a in workers:
        ans_label.append(class_dic[answers[a]])
    return ans_label
def label2num(answers, class_dic, class_dic_r):
    ans_return = copy.deepcopy(answers)
    if answers[0][0] in class_dic:
        for i in range(len(answers)):
            for w in range(len(answers[i])):
                ans_return[i][w] = class_dic[answers[i][w]]
    elif answers[0][0] in class_dic_r:
        for i in range(len(answers)):
            for w in range(len(answers[i])):
                ans_return[i][w] = class_dic_r[int(answers[i][w])]
    else: print("error")
    return ans_return

In [56]:
#ワーカのタスクに対する回答の事後確率の初期化
#Algorithm 1のInitialixe posterior distribution using weighted mv(論文p6)
def init_posdis(redundancy, ans_m, k):
    simple_agg = np.zeros((num_words,k))
    mv_ans = np.zeros((num_words))
    for r in range(redundancy):
        simple_agg += (1/redundancy)*ans_m[r]
    for i in range(len(simple_agg)):
        mv_ans[i] = np.argmax(simple_agg[i])
    return mv_ans
def ans_matrix(ans_train, redundancy, k):
    ans_m = np.zeros((num_words, redundancy,k))
    m = 0
    for n in range(len(ans_train)):
        for i in range(len(ans_train[n][0])):
            ans_m[m] = np.eye(k)[ans_train[n][2][i][:5]]
            m += 1
    return ans_m
#deep-learining-modelに渡すデータの書き込み
def write_prediction(est_labels, ans_train, iter_num):
    w_list = []
    for n in range(len(ans_train)):
        for i in range(len(ans_train[n][0])):
            w_list.append(ans_train[n][0][i] +" "+ str(est_labels[n][i]) + "\n")
        w_list.append("\n")
    w_file = path_w + "/prediction.txt"
    with open(w_file, mode='w') as f:
        f.writelines(w_list)
    with open("./MBEM_outputs/prediction"+str(iter_num)+".txt", mode='w') as f:
        f.writelines(w_list)
#deep-learning-modelからのデータの読み込み
def get_newest_prediction():
    files_list = glob.glob("./deep-learning-model/outputs/*")
    latest_file = max(glob.glob(max(files_list, key=os.path.getctime)+"/*"), key=os.path.getctime) + "/best_model_log/pred.txt"
    prediction_list = [list(l.split()) for l in open(latest_file) if l != "\n"]
    return prediction_list

In [66]:
class_list = list(set(l.split()[1] for l in open(truth_file) if l != "\n"))
class_dic = {k:i for i, k in enumerate(sorted(class_list))}
class_dic_r = {class_dic[c]:c for c in class_dic}
class_label = {'LOC': 0, 'MISC': 1, 'ORG': 2, 'PER': 3, 'O': 4}
labelnum_dic = {'B-LOC': 0, 'B-MISC': 1, 'B-ORG': 2, 'B-PER': 3, 'I-LOC': 0, 'I-MISC': 1, 'I-ORG': 2, 'I-PER': 3, 'O': 4}
#データの読み込み
ans_train, words_dic = load_crowd_data(ans_file, truth_file, train_num)
X_true, y_true = read_output('../MBEM/ner-mturk/ground_truth.txt', class_dic_r)
X_true5, y_true5 = read_output('../MBEM/ner-mturk/testset5.txt', class_dic_r)
workers_count = []
for i in range(len(ans_train)):
    for w in ans_train[i][1]:
        workers_count.append(w)
worker_num = len(set(workers_count))
class_num = len(class_dic)
#多分データを変えるごとにクラス確認したくなるので残しておく
num_words = 0
for l in ans_train:
    num_words += len(l[0])
print("worker_num", worker_num)
print("クラス：",class_dic)
print("クラス数：", class_num)
print("タスク数",len(ans_train))
print("タスク数(単語):", num_words)
print("語彙数", len(words_dic))

worker_num 47
クラス： {'B-LOC': 0, 'B-MISC': 1, 'B-ORG': 2, 'B-PER': 3, 'I-LOC': 4, 'I-MISC': 5, 'I-ORG': 6, 'I-PER': 7, 'O': 8}
クラス数： 9
タスク数 5385
タスク数(単語): 70985
語彙数 12378


In [46]:
#ans_mは回答(redundancy)×タスク(単語)×ラベル数
ans_m = ans_matrix(ans_train, redundancy, k)
ans_m = ans_m.transpose(1, 0, 2) 
#deep-learining-modelに渡すデータの書き込み
w_list = []
i = 0
simple_agg = init_posdis(redundancy, ans_m, k)
for task in ans_train:
    for word in task[0]:
        w_list.append(word +" "+ str(class_dic_r[int(simple_agg[i])]) + "\n")
        i += 1
    w_list.append("\n")
w_file = path_w + "/agg_test.txt"
with open(w_file, mode='w') as f:
    f.writelines(w_list)

In [50]:
x_agg, y_agg = read_output('./MBEM_outputs/agg_test.txt', class_dic_r, to_label=False)

In [59]:
#
est_labels = get_newest_prediction()

In [194]:
def cal_occurence(est_labels, ans_train, words_dic):
    occurence = np.zeros((J,len(words_dic)))
    for n in range(len(ans_train)):
        for t in range(len(ans_train[n][0])):
            occurence[class_dic[est_labels[n][t]],words_dic[ans_train[n][0][t]]] += 1
    return occurence
def cal_exp_log_rho(kappa, occurence):
    exp_log_rho = np.zeros((J,len(words_dic)))
    for j in range(J):
        exp_log_rho[j] = special.digamma(occurence[j]+kappa[j]) - special.digamma(np.sum(occurence+kappa,axis=0))
    return exp_log_rho
def cal_exp_log_conf(N, K, J):
    exp_log_conf = np.zeros((K,J,J,J)) 
    for k in range(K):
        for j in range(J):
            for l in range(J):
                for m in range(J):
                    exp_log_conf[k,j,l,m] = special.digamma(N[k,j,l,m]) - special.digamma(np.sum(N[k,j,l]))
    return exp_log_conf
def cal_exp_log_transe(N_s, gamma):
    exp_log_transe = np.zeros((J,J))
    exp_log_transe = special.digamma(N_s+gamma) - np.matrix(np.ones((J,J)))*special.digamma(np.sum(N_s[j]+gamma[j]))
    return exp_log_transe
def cal_ll(exp_log_conf, exp_log_rho, J, x): 
    ll = np.zeros((J))
    temp_sum = np.sum(exp_log_conf[:,:,ans_train[n]]) * rho

In [ ]:
#未完成
def cal_N(alpha, ans_train, label_post):
    temp_sum = np.zeros((K,J,J,J))
    N = np.zeros((K,J,J,J))
    ans = label2num(answers, class_dic, class_dic_r)
    for n in range(len(ans_train)):
        for t in range(len(ans_train[n][0])):
            #ここはワーカーの人数拡張に伴って変更する必要あり
            if t == 0:
                temp_sum += label_post[n,t]
            temp_sum += label_post[n,t] \
            *(np.matrix(np.identity(class_num)[ans_train[n][2][t][0]]).T*np.matrix(np.identity(class_num)[ans_train[n][2][t][0]])[:, :, np.newaxis] \
            *np.matrix(np.identity(class_num)[ans_train[n][2][t-1][0]]).T *np.matrix(np.identity(class_num)[ans_train[n][2][t-1][0]]))
    N = alpha + temp_sum
    return N
def cal_N_s(post_transe):
    pass

In [199]:
#input 
#ラベル :y
#単語: x

K = worker_num
J = class_num
#初期値を設定する
conf = np.zeros((K,J,J,J)) #worker,true_label,woker_ans(n-1),worker_ans(n)
trans_prob = np.zeros((J,J))

#0.1くらいで
alpha = 0.1 * np.ones((K,J,J,J))
kappa = 10 * np.ones((J, len(words_dic)))
gamma = 10 * np.ones((J,J))

occurence = cal_occurence(est_labels, ans_train, words_dic)
exp_log_rho = cal_exp_log_rho(kappa, occurence)

conf = np.random.dirichlet(alpha, size=(K,J,J,J)) #worker,true_label,woker_ans(n-1),worker_ans(n)
trans_prob = np.random.dirichlet(gamma, size=(J,J)) #遷移確率

label_trans_num = np.zeros((K,J,J,J))


        
def forward(exp_log_rho, exp_log_trans, x, y, J):
    n = len(y) # タスク数
    # 前向きアルゴリズム
    log_r_m = np.zeros((n, J))

    for t in range(n):
        if t == 0:
            ll[y[t]] = exp_log_rho
            log_r_m[t] = ll[y[t]]
            continue
        ll = cal_ll(exp_log_conf, exp_log_rho, J, x[t])
        temp_sum = 0
        for l in range(J):
            temp_sum += log_r_m[t-1,l] + exp_log_trans[l,j]
        log_r_m[t] = temp_sum + ll

        return log_r_m

def backward(exp_log_rho, exp_log_trans, x, y, J):
    n = len(y) 
    # 後ろ向きアルゴリズム
    log_lambda = np.zeros((n,J))
    back_lambda = np.zeros((n,J))
    ll = np.zeros((n,J))
    ll[t] = cal_ll(exp_log_conf, exp_log_rho, J, x[t])
    for a in range(n):
        t = n-1-a
        ll[t] = cal_ll(exp_log_conf, exp_log_rho, J, x[t+1])
        for l in range(J):
            log_lambda[t] += log_lambda[t+1,l] + exp_log_trans[j,l] + ll[t+1,l]
    back_lambda = np.exp(log_lambda) 

#ビタビアルゴリズム



In [ ]:
exp_log_trans = cal_exp_log_trans(N_s, gamma)

In [25]:
#input 
#ラベル :y
#単語: x

K = worker_num
J = class_num
#初期値を設定する
conf = np.zeros((K,J,J,J)) #worker,true_label,woker_ans(n-1),worker_ans(n)
rho = np.zeros((J)) #出力確率
trans_prob = np.zeros((J,J))

class VB:
    def __init__(self, K, J, alpha, kappa, gamma ):
        self.conf = np.random.dirichlet(alpha, size=(K,J,J,J)) #worker,true_label,woker_ans(n-1),worker_ans(n)
        self.rho = np.random.dirichlet(kappa, size=(J)) #出力確率
        self.trans_prob = np.random.dirichlet(gamma, size=(J,J)) #遷移確率
        
    def forward(self, y, J):
        n = len(y) # タスク数

        # 前向きアルゴリズム
        r = np.zeros((n, J))
        ll
        

        # Step2
        for t in range(1, n):
            alpha[t, :] = np.dot(alpha[t-1, :], A) * B[:, x[t]]

        # Step3
        return round(np.sum(alpha[-1]), 3)

    def backward(self, x, c):
        n = len(x) # 観測回数

        # 後ろ向きアルゴリズム
        # Step1
        beta = np.zeros((n, c))
        beta[-1, :] = 1

        # Step2
        for t in range((n-1), 0, -1):
            beta[t-1, :] = np.dot(self.A, (self.B[:, x[t]] * beta[t, :]))

        # Step3
        return round(sum(p[:] * self.B[:, x[0]] * beta[0, :]), 3)
    
    def exp_log_conf(


        
#前向き後ろ向きアルゴリズム
#論文の7式

#ビタビアルゴリズム

#

In [26]:
alpha = 10 * np.ones((K))
kappa = 10 * np.ones((J))
gamma = 10 * np.ones((J,J))
VB(K,J,alpha,kappa,gamma)

In [27]:
VB.conf[0]

AttributeError: type object 'VB' has no attribute 'conf'

In [ ]:
s_symbol = [ "w1", "w2", "w3" ] # 状態
c = len(s_symbol)               # 状態数
v_symbol = [ "v1", "v2" ]       # 出力記号
m = len(v_symbol)               # 出力数

# 遷移確率行列
A = np.array([[0.1, 0.7, 0.2], [0.2, 0.1, 0.7], [0.7, 0.2, 0.1]])
# 出力確率行列
B = np.array([[0.9, 0.1], [0.6, 0.4], [0.1, 0.9]])
# 初期確率行列
p = np.array([1/3, 1/3, 1/3])

class HMM:

    def __init__(self, p, A, B): 
        self.p = p
        self.A = A
        self.B = B

    def forward(self, x, c):
        n = len(x) # 観測回数

        # 前向きアルゴリズム
        alpha = np.zeros((n, c))

        # Step1
        alpha[0, :] = p[:] * B[:, x[0]]

        # Step2
        for t in range(1, n):
            alpha[t, :] = np.dot(alpha[t-1, :], A) * B[:, x[t]]

        # Step3
        return round(np.sum(alpha[-1]), 3)

    def backward(self, x, c):
        n = len(x) # 観測回数

        # 後ろ向きアルゴリズム
        # Step1
        beta = np.zeros((n, c))
        beta[-1, :] = 1

        # Step2
        for t in range((n-1), 0, -1):
            beta[t-1, :] = np.dot(self.A, (self.B[:, x[t]] * beta[t, :]))

        # Step3
        return round(sum(p[:] * self.B[:, x[0]] * beta[0, :]), 3)

if __name__=="__main__":
    # 観測結果
    x = [ 0, 1, 0 ] # v1, v2, v1という結果

    hmm = HMM(p, A, B)
    print(hmm.forward(x, c))
    print(hmm.backward(x, c))

In [ ]:
def post_prob_DS(est_labels, ans_train, k, worker_num, num_words):
    #学習機とラベルの更新
    n = len(ans_train)
    m = worker_num
    e_conf = np.zeros((m,k,k,k))
    temp_conf = (1/float(k))*np.ones((m,k,k,k))
    labels_md = 0.000001 * np.ones((k,k)) #正規化の時に困りそうなので0にしないでおく
    task_acc = np.zeros((n, k, k))
    #est_labelsを正解ラベルとして混同行列を作成 式(7)
    for i in range(n):
        pre_ans = ans_train[i][2][0]
        for l in range(len(ans_train[i][2])-1):
            l += 1
            #真のラベルの周辺分布を更新
            labels_md[int(est_labels[i][l-1]),int(est_labels[i][l])] += 1
            for a, j in enumerate(ans_train[i][1]): #各回答ワーカーについて
                temp_conf[j,int(est_labels[i][l]),:,:] = temp_conf[j,int(est_labels[i][l]),:,:] + np.outer(np.eye(k)[ans_train[i][2][l-1][a]],np.eye(k)[ans_train[i][2][l][a]]) #外積 混同行列の作成
    #temp_confの正規化
    for j in range(m):  
        for p in range(k):
            e_conf[j,p,:,:] = np.divide(temp_conf[j,p,:,:],np.outer(np.sum(temp_conf[j,p,:,:],axis =1),np.ones(k)))
    labels_md = np.divide(labels_md, np.sum(labels_md)*np.ones(k))
    #混同行列を使って真のラベル推定(論文p5の(5)式)
    for i in range(n):
        for l in range(len(ans_train[i][2])):
            temp_class = 0.0
            for r, a in enumerate(ans_train[i][1]):
                if l == 0:
                    w_conf = np.sum(e_conf[a,:,:,:], axis=1)
                else:
                    w_conf = e_conf[a,:,ans_train[i][2][l-1][r],:]
                #混同行列の該当列をとる
                temp_acc = np.log(np.dot(w_conf,np.transpose(np.eye(k)[ans_train[i][2][l][r]])))
                temp_class = temp_class + temp_acc
            temp_class = np.log(labels_md) +temp_class
            est_labels[i][l] = np.argmax(temp_class)%9
    return est_labels, e_conf